In [1]:
import pandas as pd
import os.path
#import textmining - breaks with float objects
from sklearn.feature_extraction.text import CountVectorizer

path = '../NYTimes_Data/'

dataSet = pd.read_csv(os.path.join(path, 'NYTimesBlogTrain.csv'))

In [2]:
dataSet.head()

,NewsDesk,SectionName,SubsectionName,Headline,Snippet,Abstract,WordCount,PubDate,Popular,UniqueID
0,Business,Crosswords/Games,NaN,More School Daze,A puzzle from Ethan Cooper that reminds me tha...,A puzzle from Ethan Cooper that reminds me tha...,508,2014-09-01 22:00:09,1,1
1,Culture,Arts,NaN,New 96-Page Murakami Work Coming in December,The Strange Library will arrive just three and...,The Strange Library will arrive just three and...,285,2014-09-01 21:14:07,0,2
2,Business,Business Day,Dealbook,Public Pension Funds Stay Mum on Corporate Expats,Public pension funds have major stakes in Amer...,Public pension funds have major stakes in Amer...,1211,2014-09-01 21:05:36,0,3
3,Business,Business Day,Dealbook,Boot Camp for Bankers,As they struggle to find new business to bolst...,As they struggle to find new business to bolst...,1405,2014-09-01 20:43:34,1,4
4,Science,Health,NaN,Of Little Help to Older Knees,Middle-aged and older patients are unlikely to...,Middle-aged and older patients are unlikely to...,181,2014-09-01 18:58:51,1,5


In [81]:
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize

#tknzr = StringTokenizer()
nltk.download('stopwords')
stop = stopwords.words('english')
# custom stopwords
#stop += ['<hashtag>', '<url>', '<allcaps>', '<number>', '<user>', '<repeat>', '<elong>', 'websummit']
dataSet["processed"] = ""

[nltk_data] Downloading package stopwords to /home/blaw/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [92]:
processed = []
for cnt, row in dataSet.iterrows():
    Abstract_Value = row['Abstract'] 
    if len(str(Abstract_Value)) > 0:
        parts = word_tokenize(str(Abstract_Value))
        clean = [i for i in parts if i not in stop]
    else:
        clean = ''
    processed.append(clean)
    #dataSet.at[cnt,'processed'] = clean

In [95]:
from sklearn.feature_extraction.text import TfidfVectorizer

ab_text = [data['Abstract'] for cnt,data in dataSet.iterrows()] # list of all tweet texts
ab_text_processed = [str.join(" ", data) for data in processed] # list of pre-processed tweet texts

In [98]:
ab_text_processed[1:2]

['The Strange Library arrive three half months Mr. Murakamis latest novel , Colorless Tsukuru Tazaki His Years Pilgrimage .']

In [99]:
vectorizer = TfidfVectorizer(min_df=4, max_features = 10000)
vz = vectorizer.fit_transform(ab_text_processed)
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))

In [104]:
#tfidf.keys()

In [103]:
print("asian: " + str(tfidf["asian"]))

asian: 7.70517999316


In [106]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=50, random_state=0)
svd_tfidf = svd.fit_transform(vz[:6532])

In [107]:
from sklearn.manifold import TSNE

tsne_model = TSNE(n_components=2, verbose=1, random_state=0)
tsne_tfidf = tsne_model.fit_transform(svd_tfidf)

[t-SNE] Computing pairwise distances...
[t-SNE] Computed conditional probabilities for sample 1000 / 6532
[t-SNE] Computed conditional probabilities for sample 2000 / 6532
[t-SNE] Computed conditional probabilities for sample 3000 / 6532
[t-SNE] Computed conditional probabilities for sample 4000 / 6532
[t-SNE] Computed conditional probabilities for sample 5000 / 6532
[t-SNE] Computed conditional probabilities for sample 6000 / 6532
[t-SNE] Computed conditional probabilities for sample 6532 / 6532
[t-SNE] Mean sigma: 0.000000
[t-SNE] Error after 100 iterations with early exaggeration: 20.508029
[t-SNE] Error after 236 iterations: 2.424675


In [114]:
#from tornado import gen
#import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show
from boken.io import output_notebook

output_notebook()
plot_tfidf = figure(plot_width=900, plot_height=700, title="NY Times - plot (tf-idf)",
    tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
    x_axis_type=None, y_axis_type=None, min_border=1)

plot_tfidf.scatter(x=tsne_tfidf[:,0], y=tsne_tfidf[:,1],
                    source=bp.ColumnDataSource({
                        "text": ab_text[:6532], 
                        "processed": ab_text_processed[:6532]
                    }))

hover = plot_tfidf.select(dict(type=HoverTool))
hover.tooltips={"text": "@text (processed: \"@processed\")"}
show(plot_tfidf)

ImportError: cannot import name gen